# 1 - Nouvelle session de Feature Engineering

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

## 1.1 - Création des fonctions

### La fonction agg_numeric

La fonction agg_numeric agrège les variables numériques d'un dataframe. Elle calcule des statistiques de base (comme la moyenne, le minimum, le maximum et la somme) pour chaque variable en question.   


In [2]:
def agg_numeric(df, group_var, df_name):
    """Agrège les valeurs numériques dans un dataframe. Cela peut
    être utilisé pour créer des caractéristiques pour chaque instance de la variable de groupement.
    
    Paramètres
    --------
        df (dataframe): 
            le dataframe sur lequel calculer les statistiques
        group_var (string): 
            la variable selon laquelle grouper le df
        df_name (string): 
            la variable utilisée pour renommer les colonnes
        
    Retour
    --------
        agg (dataframe): 
            un dataframe avec les statistiques agrégées pour 
            toutes les colonnes numériques. Chaque instance de la variable de groupement aura 
            les statistiques (moyenne, min, max, somme; actuellement supportées) calculées. 
            Les colonnes sont également renommées pour suivre les caractéristiques créées.
    
    """
    # Supprimer les variables d'identification autres que la variable de groupement
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns=col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Grouper par la variable spécifiée et calculer les statistiques
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Besoin de créer de nouveaux noms de colonnes
    columns = [group_var]

    # Itérer à travers les noms des variables
    for var in agg.columns.levels[0]:
        # Ignorer la variable de groupement
        if var != group_var:
            # Itérer à travers les noms des statistiques
            for stat in agg.columns.levels[1][:-1]:
                # Créer un nouveau nom de colonne pour la variable et la statistique
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg


### Fonction pour traiter les variables catégorielles

Voici maintenant une fonction pour gérer les variables catégorielles. Cette fonction prendra la même forme que la fonction agg_numeric, c'est-à-dire qu'elle acceptera un datafrale et une variable de regroupement. Elle calculera ensuite les effectifs et les effectifs normalisés de chaque catégorie pour toutes les variables catégorielles de la base de données.

In [3]:
def count_categorical(df, group_var, df_name):
    """Calcule les décomptes et les décomptes normalisés pour chaque observation
    de `group_var` de chaque catégorie unique dans chaque variable catégorielle.
    
    Paramètres
    --------
    df : dataframe 
        Le dataframe pour lequel calculer les décomptes de valeurs.
        
    group_var : string
        La variable selon laquelle grouper le dataframe. Pour chaque valeur unique
        de cette variable, le dataframe final aura une ligne.
        
    df_name : string
        Variable ajoutée devant les noms de colonnes pour suivre les colonnes.

    Retour
    --------
    categorical : dataframe
        Un dataframe avec les décomptes et les décomptes normalisés de chaque catégorie unique dans chaque variable catégorielle,
        avec une ligne pour chaque valeur unique de `group_var`.
        
    """
    
    # Sélectionner les colonnes catégorielles
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # S'assurer de mettre l'identifiant sur la colonne
    categorical[group_var] = df[group_var]

     # Grouper par la variable de groupe et calculer la somme et la moyenne
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
     # Itérer à travers les colonnes du niveau 0
    for var in categorical.columns.levels[0]:
        # Itérer à travers les statistiques du niveau 1
        for stat in ['count', 'count_norm']:
            # Créer un nouveau nom de colonne
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

## 1.2 - Feature Engineering sur les dataframe 'bureau' et 'bureau_balance'

Nous disposons à présent de tous les éléments nécessaires pour intégrer les informations relatives aux prêts antérieurs contractés auprès d'autres institutions et les informations relatives aux paiements mensuels de ces prêts dans le dataframe principal.

* **application_train/application_test**: the main training and testing data avec des informations sur chaque demande de prêt chez Home Credit. Chaque prêt a sa propre ligne et est identifié par la caractéristique `SK_ID_CURR`. Les données du train sont accompagnées de la  `TARGET` qui indique 0 : le prêt a été remboursé ou 1 : le prêt n'a pas été remboursé. 
* **bureau**: les données concernant les crédits antérieurs du client auprès d'autres institutions financières. Chaque crédit antérieur a sa propre ligne dans le bureau, mais un prêt dans les données de la demande peut avoir plusieurs crédits antérieurs.
* **bureau_balance**: données mensuelles sur les crédits antérieurs dans le bureau. Chaque ligne correspond à un mois d'un crédit antérieur, et un seul crédit antérieur peut avoir plusieurs lignes, une pour chaque mois de la durée du crédit. 

In [4]:
train = pd.read_csv('app_train_domain.csv')
test = pd.read_csv('app_test_domain.csv')
bureau = pd.read_csv('bureaux.csv')
bureau_balance = pd.read_csv('bureau_balances.csv')

### Application de la fonction count_categorical sur le 'bureau'

In [5]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


### Application de la fonction agg_numeric sur le 'bureau'

In [6]:
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


### Application de la fonction count_categorical sur le 'bureau_balance'

In [7]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_2_count,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_3_count,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


### Application de la fonction agg_numeric sur le 'bureau_balance'

In [8]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum
0,5001709,97,-48.0,0,-96,-4656
1,5001710,83,-41.0,0,-82,-3403
2,5001711,4,-1.5,0,-3,-6
3,5001712,19,-9.0,0,-18,-171
4,5001713,22,-10.5,0,-21,-231


### Statistiques agrégées du 'bureau_balance' par client

In [9]:
# regroupement par prêt
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge pour inclure le SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

# Aggrégation des statspour chaque  client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

### Merge avec le train et le test

In [10]:
original_features_train= list(train.columns)
print('Original Number of Features: ', len(original_features_train))

Original Number of Features:  127


In [11]:
original_features_test = list(test.columns)
print('Original Number of Features: ', len(original_features_test))

Original Number of Features:  126


In [12]:
# Merge with the value counts of bureau
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')


# Merge with the stats of bureau
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')


# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')


In [13]:
train.shape, test.shape

((307511, 338), (48744, 337))

In [14]:
train.to_csv('train_after_bureau.csv', index=False)

In [15]:
test.to_csv('test_after_bureau.csv', index=False)